In [1]:
# size dist plus growth rates
import numpy as np

import matplotlib.pyplot as plt
import seaborn
% matplotlib inline
from scipy.stats import norm

import pandas as pd

In [5]:

data_path = './../../data/processed/'

df = pd.read_csv(data_path + 'ID_Y.csv')
df = df.loc[df.IMPORT == 0]

cuadro = df.set_index(['ID', 'YEAR'])[['VART']].unstack()

# Firms present in at least some time periods
cuadro = cuadro.loc[cuadro.count(1) == 17]
cuadro_rolled = cuadro

annual_logdiff = np.log10(cuadro_rolled).diff(1, axis = 1).replace(np.inf, np.nan).replace(-np.inf, np.nan).dropna(how = 'all')
annual_log = np.log10(cuadro_rolled.reindex(annual_logdiff.index))

annual_log.columns = annual_log.columns.set_levels(annual_log.columns.levels[0].str.replace('VART', 'annual_log'), level=0)
annual_logdiff.columns = annual_logdiff.columns.set_levels(annual_logdiff.columns.levels[0].str.replace('VART', 'annual_logdiff'), level=0)

df_concat = pd.concat([annual_log, annual_logdiff], axis = 1)

df_level_g = df_concat.stack()

In [143]:
# g_rates = df_level_g

# g_rates = g_rates.reset_index()
# g_rates['YEAR_diff'] = g_rates['YEAR'].diff()
# # g_rates = g_rates.loc[g_rates['YEAR_diff'] == 1].set_index(['ID', 'YEAR'])

# g_rates['pair'] = np.arange(len(g_rates))//2
# valid_pairs = g_rates.groupby('pair')['annual_logdiff'].count() == 2
# valid_pairs = valid_pairs.index[valid_pairs].tolist()

# g_rates = g_rates.loc[g_rates.pair.isin(valid_pairs)]

# g_rates_info = g_rates.groupby('pair').agg({'annual_log': 'mean', 'annual_logdiff': ['first', 'last']})
# g_rates_info.columns = ['level', 'logdiff1', 'logdiff2']
# g_rates_info = g_rates_info.sort_values(by = 'level').reset_index(drop = True)

# g_rates_info.to_csv('./../../data/processed/g_rates_info.csv', index = False)

In [16]:
df_growth = df_level_g[['annual_logdiff']].unstack().dropna(axis = 1)
df_growth['mean_annual_log'] = df_level_g[['annual_log']].unstack().mean(1)
df_growth = df_growth.sort_values(by = 'mean_annual_log')

df_growth.to_csv('./../../data/processed/g_rates_16_info.csv')